**Troubleshooting Scripts - Execution-Related DMVs (sys.dm\_exec\_requests, sys.dm\_os\_waiting\_tasks, etc)**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Get the list of active requests in the system**

The script filters out SPID \<= 50 to get user sessions only. This may miss Service Broker activation procedures. Also, it is possible to have system sessions with session\_id \> 50 nowadays. You can also change the script and use sys.dm\_exec\_sessions.is\_user\_process instead. 

Sort based on your troubleshooting strategy. For example, by cpu\_time, if you look for the most CPU-intensive queries, (logical\_reads + writes) for I/O intensive queries, etc

In [ ]:
SELECT	
    er.session_id
    ,er.request_id
    ,DB_NAME(er.database_id) as [database]
    ,er.start_time
    ,CONVERT(DECIMAL(21,3),er.total_elapsed_time / 1000.) AS [duration]
    ,er.cpu_time
    ,SUBSTRING(
        qt.text, 
        (er.statement_start_offset / 2) + 1,
            ((CASE er.statement_end_offset
                WHEN -1 THEN DATALENGTH(qt.text)
                ELSE er.statement_end_offset
            END - er.statement_start_offset) / 2) + 1
    ) AS [statement]
    ,er.status
    ,er.wait_type
    ,er.wait_time
    ,er.wait_resource
    ,er.blocking_session_id
    ,er.last_wait_type
    ,er.reads
    ,er.logical_reads
    ,er.writes
    ,er.granted_query_memory
    ,er.dop -- SQL Server 2016+
    ,er.row_count
    ,er.percent_complete
    ,es.login_time
    ,es.original_login_name
    ,es.host_name
    ,es.program_name
    ,c.client_net_address
    ,ib.event_info AS [buffer]
    ,qt.text AS [sql]
    ,TRY_CONVERT(XML,p.query_plan) as [query_plan]
FROM	
    sys.dm_exec_requests er WITH (NOLOCK)
        OUTER APPLY sys.dm_exec_input_buffer(er.session_id, er.request_id) ib
        OUTER APPLY sys.dm_exec_sql_text(er.sql_handle) qt
        OUTER APPLY 
            sys.dm_exec_text_query_plan
            (
                er.plan_handle
                ,er.statement_start_offset
                ,er.statement_end_offset
            ) p
        LEFT JOIN sys.dm_exec_connections c WITH (NOLOCK) ON 
            er.session_id = c.session_id 
        LEFT JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON 
            er.session_id = es.session_id
WHERE
    er.status <> 'background' 
    AND er.session_id > 50
ORDER BY 
    er.cpu_time desc
OPTION (RECOMPILE, MAXDOP 1);

**Get the list of active requests in the system using lightweight profiling instead of regular query plan**

Requires SQL Server 2016 SP1+. See [documentation](https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-query-statistics-xml-transact-sql) for more details.

In [ ]:
SELECT	
    er.session_id
    ,er.request_id
    ,DB_NAME(er.database_id) as [database]
    ,er.start_time
    ,CONVERT(DECIMAL(21,3),er.total_elapsed_time / 1000.) AS [duration]
    ,er.cpu_time
    ,SUBSTRING(
        qt.text, 
        (er.statement_start_offset / 2) + 1,
        ((CASE er.statement_end_offset
            WHEN -1 THEN DATALENGTH(qt.text)
            ELSE er.statement_end_offset
        END - er.statement_start_offset) / 2) + 1
    ) AS [statement]
    ,er.status
    ,er.wait_type
    ,er.wait_time
    ,er.wait_resource
    ,er.blocking_session_id
    ,er.last_wait_type
    ,er.reads
    ,er.logical_reads
    ,er.writes
    ,er.granted_query_memory
    ,er.dop
    ,er.row_count
    ,er.percent_complete
    ,es.login_time
    ,es.original_login_name
    ,es.host_name
    ,es.program_name
    ,c.client_net_address
    ,ib.event_info AS [buffer]
    ,qt.text AS [sql]
    ,p.query_plan
FROM	
    sys.dm_exec_requests er WITH (NOLOCK)
        OUTER APPLY sys.dm_exec_input_buffer(er.session_id, er.request_id) ib
        OUTER APPLY sys.dm_exec_sql_text(er.sql_handle) qt
        OUTER APPLY sys.dm_exec_query_statistics_xml(er.session_id) p
        LEFT JOIN sys.dm_exec_connections c WITH (NOLOCK) ON 
            er.session_id = c.session_id 
        LEFT JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON 
            er.session_id = es.session_id
WHERE
    er.status <> 'background'
    AND er.session_id > 50
ORDER BY 
    er.cpu_time desc
OPTION (RECOMPILE, MAXDOP 1);


**Schedulers information**

Pay attention to the number of online schedulers per NUMA node. Uneven distribution will impact system performance

In [ ]:
SELECT 
    parent_node_id AS [NUMA Node]
    ,COUNT(*) AS [Schedulers]
    ,SUM(IIF(status = N'VISIBLE ONLINE',1,0))
        AS [Online Schedulers]
    ,SUM(IIF(status = N'VISIBLE OFFLINE',1,0))
        AS [Offline Schedulers]
    ,SUM(current_tasks_count) 
        AS [Current Tasks] 
    ,SUM(runnable_tasks_count) 
        AS [Runnable Tasks] 
FROM 
    sys.dm_os_schedulers WITH (NOLOCK)
WHERE 
    status IN (N'VISIBLE ONLINE',N'VISIBLE OFFLINE') 
GROUP BY 
    parent_node_id
OPTION (RECOMPILE, MAXDOP 1);

**The list of currently waiting tasks**

In some cases, you may want to remove filter by session\_id \> 50 to get the information about system sessions

In [ ]:
SELECT
	wt.session_id
	,wt.wait_type
	,wt.wait_duration_ms
	,wt.blocking_session_id
	,wt.resource_description
FROM 
	sys.dm_os_waiting_tasks wt WITH (NOLOCK)
WHERE
	wt.session_id > 50
ORDER BY
	wt.wait_duration_ms DESC
OPTION (MAXDOP 1, RECOMPILE);

**Get connection information for the sessions**

Set the session\_id in WHERE clause if you want to troubleshoot the single session/connection

In [ ]:
SELECT
	ec.session_id
	,DB_NAME(s.database_id) AS [database] -- SQL Server 2012+
	,s.login_time 
	,s.host_name
	,s.program_name
	,s.login_name
	,s.original_login_name
	,s.cpu_time
	,s.last_request_start_time
	,s.reads
	,s.writes
	,ec.connect_time
	,qt.text AS [SQL]
FROM 
	sys.dm_exec_connections ec WITH (NOLOCK)
		LEFT JOIN sys.dm_exec_sessions s WITH (NOLOCK) ON
			ec.session_id = s.session_id
		OUTER APPLY
			sys.dm_exec_sql_text(ec.most_recent_sql_handle) qt
--WHERE
--	ec.session_id = 51 -- session id of the session
ORDER BY
    session_id
OPTION (MAXDOP 1, RECOMPILE)

**Waiting tasks with connection information**

In [ ]:
SELECT
	wt.session_id
	,DB_NAME(s1.database_id) AS [database] -- SQL Server 2012+
	,wt.wait_type
	,wt.wait_duration_ms
	,wt.blocking_session_id
	,wt.resource_description
	,s2.login_time AS [blocking_login_time]
	,s2.host_name AS [blocking_host_name]
	,s2.program_name AS [blocking_program_name]
	,s2.login_name AS [blocking_login_name]
	,s2.original_login_name AS [blocking_original_login]
	,s2.cpu_time AS [blocking_cpu_time]
	,s2.last_request_start_time AS [blocking_request_start_time]
	,s2.reads AS [blocking_reads]
	,s2.writes AS [blocking_writes]
	,ec2.connect_time AS [blocking_connect_time]
	,qt2.text AS [blocking_sql]  
    ,s1.login_time AS [blocked_login_time]
	,s1.host_name AS [blocked_host_name]
	,s1.program_name AS [blocked_program_name]
	,s1.login_name AS [blocked_login_name]
	,s1.original_login_name AS [blocked_original_login]
	,s1.cpu_time AS [blocked_cpu_time]
	,s1.last_request_start_time AS [blocked_request_start_time]
	,s1.reads AS [blocked_reads]
	,s1.writes AS [blocked_writes]
	,ec1.connect_time AS [blocked_connect_time]
	,qt1.text AS [blocked_sql] 
FROM 
	sys.dm_os_waiting_tasks wt WITH (NOLOCK)
	    JOIN sys.dm_exec_connections ec1 WITH (NOLOCK) ON
            wt.session_id = ec1.session_id
		LEFT JOIN sys.dm_exec_sessions s1 WITH (NOLOCK) ON
			ec1.session_id = s1.session_id
		OUTER APPLY
			sys.dm_exec_sql_text(ec1.most_recent_sql_handle) qt1    
	    LEFT JOIN sys.dm_exec_connections ec2 WITH (NOLOCK) ON
            wt.blocking_session_id = ec2.session_id
		LEFT JOIN sys.dm_exec_sessions s2 WITH (NOLOCK) ON
			ec2.session_id = s2.session_id
		OUTER APPLY
			sys.dm_exec_sql_text(ec2.most_recent_sql_handle) qt2               
WHERE
	wt.session_id > 50
ORDER BY
	wt.wait_duration_ms DESC
OPTION (MAXDOP 1, RECOMPILE);